## Создание приложения со streamlit

### Установка streamlit

**Важный момент:** чтобы пользоваться всеми возможностями библиотеки `streamlit`, нужна версия Python не ниже 3.8. Если у вас Python более старой версии, обновлять его необязательно, можно запускать приложения streamlit в Google Colab (см. ниже).

Проверим свою версию Python:

In [ ]:
!python --version

Установим `streamlit` и принудительно обновим (опция `--upgrade`) на случай, если библиотека уже есть, но в старой версии:

In [ ]:
!pip install streamlit --upgrade

Импортируем библиотеку:

In [ ]:
import streamlit as st

Если импортируется с некоторым небольшим предупреждением о  `numexpr`, нестрашно, если вылетает ошибка `ImportError`, нужно разобраться с тем, какие версии каких модулей оказались несовместимыми (обычно проблема с теми инструментами, которые общие и у `streamlit`, и у `pandas`). Пример проблемного импорта ниже:

В примере выше проблема с `numexpr` и `bottleneck`, их можно обновить отдельно:

In [ ]:
!pip install numexpr --upgrade

In [ ]:
!pip install bottleneck --upgrade

Или установить конкретную версию модуля/библиотеки, которая требуется, согласно сообщению об ошибке:

In [ ]:
!pip install numexpr==2.8.4

А теперь снова попробовать импортировать `streamlit`:

In [ ]:
import streamlit as st

### Проверка streamlit

Если со `streamlit` работаем в первый раз, можно запустить встроенный в него пример (не наш код) с командной строки, иначе в Jupyter ничего не получится – при первичном вызове `streamlit` ожидается ввод email с клавиатуры. 

В Windows можно зайти в консоль Anaconda (в меню *Пуск* ищем папку *Anaconda*, в ней терминал/командную строку). На Mac просто запускаем терминал (в *Launchpad* ищем Терминал, открываем новое окно, не нужно что-то вводить что-то в черное/белое окно, которое используется для запуска работающего Jupyter).

В терминале/командной строке пишем:

    streamlit hello
    
Нажимаем *Enter*, если запрашивается email, вводим его и нажимаем *Enter* (или просто нажимаем на *Enter*, если не хотим вводить email).

Пример окна консоли после успешного запуска:

<img src="one.jpeg" width="70%">

Запускается шаблонное приложение, иллюстрирующее возможности этой библиотеки. Оно открывается в новой вкладке браузера, локально, обычно по адресу `http://localhost:8501/`. Эта ссылка является активной только на текущем ноутбуке, пока приложение работает, то есть пока окно с консолью не закрыто (исполнение ячейки в Jupyter не остановлено).

###  Создание и запуск приложения

Создадим приложение, которое на основе данных по заработной плате сотрудников университетов США в файле `Salaries.csv` выводит описательные статистики для выбранной пользователем переменной и строит график для визуализации её распределения:

* если тип столбца с переменной текстовый (`object`), строится столбиковая диаграмма;
* если тип столбца с переменной числовой (иное), строится гистограмма.

Создадим новый исполняемый файл с расширением `.py`, впишем туда код и сохраним изменения. 

**Способ 1.** В Home Page в Jupyter Notebook выбираем *New - Text file*, изменяем расширение `.txt` на `.py` и даем файлу название типа `myapp.py`. Вписываем код (пример ниже без строки с `%%writefile myapp.py`) и сохраняем изменения.

**Способ 2.** В файле Jupyter Notebook в ячейку помещаем код, который должен быть в файле `myapp.py`, в начале ячейки добавляем строку `%%writefile myapp.py`, чтобы содержимое ячейки записалось и сохранилось в файл.

In [1]:
%%writefile myapp.py

import streamlit as st
import pandas as pd
from matplotlib import pyplot as plt

# prepare data

dat = pd.read_csv("Salaries.csv")
dat.rename(columns = {"yrs.since.phd" : "phd", 
            "yrs.service" : "service"}, inplace = True)

# prepare the list of variables,
# without Untitled, the 1st one

options = list(dat.columns[1:]) 

# add title and a dropdown menu to select a column

st.title("Salaries in US universities")
selected = st.sidebar.selectbox("Choose a variable", options)

# add a field for the color of the graph (e.g. #346eeb)

fill_color = st.sidebar.text_input("Enter a color:", value = "#346eeb")

# create a table with descriptive statistics

selected_table = dat[selected].describe()

# if column is not numeric (type object)
# create a table with frequencies
# and plot a bar chart -> plot in ax, save as fig
# otherwise, plot a histogram -> plot in ax, save as fig

if dat[selected].dtype == "object":
    freqs = dat[selected].value_counts()
    fig, ax = plt.subplots()
    ax.bar(freqs.index, freqs.values, color = fill_color)
    
else:
    fig, ax = plt.subplots()
    ax.hist(dat[selected], color = fill_color, edgecolor = "white")
    
# add two tabs, 1st with table of descriptives,     
# 2nd with the graph

tab1, tab2 = st.tabs(["Statistics", "Visualization"])

with tab1:
    st.table(selected_table)

with tab2:    
    st.pyplot(fig)
    

Overwriting myapp.py


Пояснения к коду:
    
* считываем данные из файла `Salaries.csv`, переименовываем столбцы, записываем названия столбцов в список строк `options`;
* добавляем заголовок (`title`);
* добавляем как элемент бокового меню (`sidebar`) выпадающее меню `selectbox` для выбора названий столбцов; 
* добавляем как элемент бокового меню (`sidebar`) поле с текстовым вводом `text_input` для выбора цвета (значение цвета `#346eeb` зафиксировано по умолчанию);
* создаем таблицу с описательными статистиками по выбранному столбцу `selected_table`;
* проверяем условие на тип столбца и отрисовываем в осях `ax` либо столбиковую диаграмму (`bar`), либо гистограмму (`hist`);
* добавляем две вкладки `tab1` и `tab2` на странице с приложением;
* во вкладке `tab1` размещаем таблицу с описательными статистиками `selected_table`;
* во вкладке `tab2` размещаем график `fig` (в нем ранее сохранена картинка либо с гистограммой, либо со столбиковой диаграммой).

Запускаем приложение! Можно запускать его через терминал/командную строку, прописывая полный  путь в файлу `myapp.py`, а можно прямо через Jupyter. 

**Через терминал/командную строку:**

Находим путь к текущей папке:

In [2]:
import os
os.getcwd()

'/Users/allat/Desktop'

В командной строке набираем 

    streamlit run "/Users/allat/Desktop/myapp.py"

и нажимаем *Enter*.

**Через Jupyter Notebook:**

В ipynb-файле рядом с `myapp.py` запускаем ячейку:

In [ ]:
!streamlit run app.py

В новой вкладке должна открыться страница с приложением, см. [видео](https://www.dropbox.com/scl/fi/9ieyqoctmahe1kvxqhrj3/st-example.mp4?rlkey=6oz350hrnnl2r3n9g159zw416&dl=0) с иллюстрацией работы.

### Запуск приложения в Google Colab

Запускать приложения в Google Colab потенциально проблематично, так как запускаться они будут где-то на облаке, к которому за рамками Colab у нас доступа нет (ссылка вида `http://localhost:8501/` ни к чему не приведет).

Однако есть способ, [описанный](https://discuss.streamlit.io/t/how-to-launch-streamlit-app-from-google-colab-notebook/42399) в самом сообществе Streamlit. 

**Шаг 1.** Заходим в Google Colab, создаем новый ноутбук, устанавливаем библиотеку `localtunnel`, она позволит запускать приложение на удаленном сервере и иметь доступ к нему:

    !npm install localtunnel
    
**Шаг 2.** Запрашиваем IP адрес для этого соединения:

In [3]:
import urllib
print("Password/Enpoint IP for localtunnel is:",
      urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 98.98.166.186


**Шаг 3.** Устанавливаем `streamlit`:

    !pip install streamlit

**Шаг 4.** Загружаем файл  с приложением в Colab или записываем код в новый исполняемый через `%%writefile` как выше. Запускаем его с фоновом режиме (чтобы оно запустилось и чтобы ячейка не «висела»):

    !streamlit run myapp.py &>/content/logs.txt&
    
**Шаг 5.** Переходим по нужному адресу `localtunnel`:

    !npx localtunnel --port 8501
    
Проходим по ссылке и вводим в поле для пароля/IP значение, которое было выдано ранее в `urlopen`, нажимаем *Click to submit*. Через некоторое время должна открыться страница с приложением.

### Публикация приложения streamlit

1. Создаем аккаунт на [Github](https://github.com/), бесплатно, теперь требует двухфакторной аутентификации.

2. Создаем аккаунт на [Streamlit](https://streamlit.io/), в нем – репозиторий (папку) с файлами для приложения и публикуем приложение по [инструкции](https://carpentries-incubator.github.io/python-interactive-data-visualizations/08-publish-your-app/index.html). 

Пример репозитория с приложением – [ссылка](https://github.com/allatambov/steamlit-salaries?tab=readme-ov-file). В папке есть исполняемый файл `.py`, файл с данными, файл `requirements.txt` с перечнем необходимых библиотек, файл `README.md` с документацией. Подробнее о файлах см. в README, обратите особое внимание на `requirements.txt`, без него на сервере ничего не заработает.